In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from scipy.fft import fft, fftfreq

# Functions

## Reading and preparing dataframe

In [ ]:
def read_clear_df(filepath, skiprows=None, delimiter=',', columns_num=None):
    """ A function for reading a dataset from a csv file and preparing the dataframe for work 
    (removing unnecessary columns, converting measurement results to float type)
    Args:
        1) filepath - the path to the csv file;
        2) skiprows - the number of first rows in the csv file to skip;
        3) delimiter - the character that separates the columns of the dataset;
        4) columns_num - the number of first columns that should be in the dataframe (used to remove 'Unnamed' columns).
    Returns:
        df - a dataframe read from a file containing only the columns necessary for operation, and the measurement results cast to float type. 
    """
    df = pd.read_csv(filepath, skiprows=skiprows, delimiter=delimiter)
    # Drop unnecessary columns
    if (columns_num != None) and (len(df.columns) > columns_num):
        for column in df.columns[columns_num:]:
            df.drop([column], axis=1, inplace=True)
            
    # Convert measurement columns to the float type
    for column in df.columns[1:]:    
        df[column] = df[column].apply(lambda measurement_str: float(measurement_str.replace(',', '.')))
        
    return df

## Visualising results

In [ ]:
def display_three_axes(df, x, y, title=None, x_label=None, y_label=None):
    """ Function for visualization of three axes (X, Y, Z) of the measurement results of the device
    Args:
        1) df - a dataframe containing the results of the device measurement;
        2) x - the name of the column of the dataframe that contains the data for the OX axis (for the three lines, this is the same data);
        3) y - the list containing the dataframe column names corresponding to the OX, OY, and OZ axis measurements, respectively;
        4) title - title of the graph;
        5) x_label - the name of the OX axis of the graph;
        6) y_label - the name of the OY axis of the graph.
    Returns:
        Nothing, but plots graph
    """
    plt.figure(figsize=(12, 6))
    plt.plot(df[x], df[y[0]], label='X')
    plt.plot(df[x], df[y[1]], label='Y')
    plt.plot(df[x], df[y[2]], label='Z')
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.show()
    

def display_three_axes_sep(df, x, y, title=None, x_label=None, y_labels=None):
    """ Function for visualization of three axes (X, Y, Z) of the measurement results of the device, a separate graph for each axis
    Args:
        1) df - a dataframe containing the results of the device measurement;
        2) x - the name of the column of the dataframe that contains the data for the OX axis (for the three lines, this is the same data);
        3) y - the list containing the dataframe column names corresponding to the OX, OY, and OZ axis measurements, respectively;
        4) title - title of the graph;
        5) x_label - the name of the OX axis of the graph;
        6) y_labels - the list of names of the OY axis of the graph.
    Returns:
        Nothing, but plots graph
    """
    fig, axes = plt.subplots(3, 1, figsize=(10, 18))
    axes[0].plot(df[x], df[y[0]], color='blue')
    axes[0].set_xlabel(x_label)
    axes[0].set_ylabel(y_labels[0])
    axes[0].set_title(title + ' accX')

    axes[1].plot(df[x], df[y[1]], color='yellow')
    axes[1].set_xlabel(x_label)
    axes[1].set_ylabel(y_labels[1])
    axes[1].set_title(title + ' accY')

    axes[2].plot(df[x], df[y[2]], color='green')
    axes[2].set_xlabel(x_label)
    axes[2].set_ylabel(y_labels[2])
    axes[2].set_title(title + ' accZ')

    plt.show()

## Exploring measurement time and frequency stability

In [ ]:
def get_measurement_time_df(df):
    """ A function for building a dataframe that contains information about the time of measurement of each reading of the device
    Args:
        1) df - a dataframe that contains a 'time' column
    Returns:
        time_measurement_df - dataframe that contains information about the time of measurement of each reading of the device
    """
    period_dict = {'start_time': [], 'end_time': [], 'measurement_time': []}
    prev_time = None
    curr_time = None
    for index, row in df.iterrows():
        if(index == len(df)):
            break
            
        prev_time = curr_time
        curr_time = row['time']

        if prev_time != None:
            period_dict['start_time'].append(prev_time)
            period_dict['end_time'].append(curr_time)
            period_dict['measurement_time'].append(curr_time - prev_time)
    
    time_measurement_df = pd.DataFrame.from_dict(period_dict)
    return time_measurement_df


def display_time_distribution(df, x):
    """ A function that is designed to display the measurement time distribution using a histogram
    Args:
        1) df - a dataframe that contains a column with measurement time data
        2) x - name of measurement time column
    """
    # Plot the measurement time distribution using seaborn's histogram
    # plt.figure(figsize=(15, 6)) - original size
    plt.figure(figsize=(25, 6))
    sns.histplot(df[x], bins=60, kde=True, color='blue')
    plt.xlabel('Period, s')
    plt.ylabel('Count')
    plt.title('Measurement Time Distribution')
    plt.xticks([i/1000 for i in range(0, 38, 1)])

    plt.show()


def display_data_collection_stability(df):
    """A function that displays the stability of data collection relative to the time when the measurements were taken and 
       relative to the measurement number
    Args:
        1) df - time_measurement_df
    """
    fig, axes = plt.subplots(2, 1, figsize=(15, 14))
    axes[0].plot(range(len(df[df['lost_data'] == 0])), df[df['lost_data'] == 0]['measurement_time'])
    axes[0].set_title(f'Stability of Data Collection (Filtered)\nAverage frequency: {freq:.3f} Hz')
    axes[0].set_xlabel('number of measurement')
    axes[0].set_ylabel('measurement time')
    
    axes[1].plot(df[df['lost_data'] == 0]['start_time'], df[df['lost_data'] == 0]['measurement_time'])
    axes[1].set_title(f'Stability of Data Collection (Filtered)\nAverage frequency: {freq:.3f} Hz')
    axes[1].set_xlabel('time of measurement')
    axes[1].set_ylabel('measurement time')
    
    plt.show()

    
def display_data_collection_stability_measurement_number(df):
    """A function that displays the stability of data collection relative to the measurement number
    Args:
        1) df - time_measurement_df
    """
    plt.figure(figsize=(12, 3))
    plt.plot(range(len(df[df['lost_data'] == 0])), df[df['lost_data'] == 0]['measurement_time'])
    plt.title(f'Stability of Data Collection (Filtered)\nAverage frequency: {freq:.3f} Hz')
    plt.xlabel('number of measurement')
    plt.ylabel('measurement time, s')
    
    plt.show()
    

def display_data_collection_stability_measurement_time(df):
    """A function that displays the stability of data collection relative to the time when the measurements were taken
    Args:
        1) df - time_measurement_df
    """
    plt.figure(figsize=(12, 3))
    plt.plot(time_measurement_df[time_measurement_df['lost_data'] == 0]['start_time'], 
             time_measurement_df[time_measurement_df['lost_data'] == 0]['measurement_time'])
    plt.title(f'Stability of Data Collection (Filtered)\nAverage frequency: {freq:.3f} Hz')
    plt.xlabel('time of measurement')
    plt.ylabel('measurement time')
    
    plt.show()
    

def display_data_collection_losses_measurement_number(df):
    """A function that displays the loses of data collection relative to the measurement number
    Args:
        1) df - time_measurement_df
    """
    plt.figure(figsize=(12, 3))
    plt.plot(range(len(df[df['lost_data'] == 1])), df[df['lost_data'] == 1]['measurement_time'])
    plt.title(f'Data Collection Losses (Filtered)\nAverage frequency: {freq:.3f} Hz')
    plt.xlabel('number of measurement')
    plt.ylabel('measurement time, s')
    
    plt.show()
    

def display_data_collection_losses_measurement_time(df):
    """A function that displays data collection losses relative to the time when the measurements were taken
    Args:
        1) df - time_measurement_df
    """
    plt.figure(figsize=(12, 3))
    plt.plot(time_measurement_df[time_measurement_df['lost_data'] == 1]['start_time'], 
             time_measurement_df[time_measurement_df['lost_data'] == 1]['measurement_time'])
    plt.title(f'Data Collection Losses (Filtered)\nAverage frequency: {freq:.3f} Hz')
    plt.xlabel('time of measurement, s')
    plt.ylabel('measurement time, s')
    
    plt.show()

## Frequency analysis of the signal spectrum

In [ ]:
def create_activities_df(path):
    """ A function for creating a dataframe containing information about the beginning, end and name of each physical activity
    Args:
        1) path - the path to the Label Studio file that contains the labeled signals
    Returns:
        Dataframe containing information about the beginning, end and name of each physical activity
    """
    labeled_activities = pd.read_csv(path)
    labels = labeled_activities['label']
    labels = np.array(labels)[0]    # the original type of labels is pd.Series
    labels = labels.split('},')
    
    activities_dict = {'start_time': [], 'end_time': [], 'activity': []}
    for label in labels:
        start_time, end_time, _, activity = label.split(',')
        activities_dict['start_time'].append(float(start_time.split(':')[1]))
        activities_dict['end_time'].append(float(end_time.split(':')[1]))
        activities_dict['activity'].append(activity.split(':')[1].strip('[]}"'))
    
    activities_df = pd.DataFrame.from_dict(activities_dict)
    activities_df = activities_df.sort_values(by='start_time').reset_index(drop=True)
    return activities_df

def add_activity_column(path, df, df_merge_col):
    """ Function to add 'activity' column to dataframe df
    Args:
        1) path - the path to the Label Studio file that contains the labeled signals
        2) df - a dataframe that contains the results of accelerometer and gyroscope measurements
        3) df_merge_col - the name of the column of the dataframe df, which contains data about the measurement time ('time' column)
    """
    activities_df_local = create_activities_df(path)
    merged_df = pd.merge_asof(df, activities_df_local, left_on=df_merge_col, right_on='start_time', direction='backward')
    df['activity'] = merged_df['activity']

In [ ]:
def show_freq_distribution(df_arg, signal_column, sampling_rate, is_divided_by_g=False):
    """ A function for displaying a frequency distribution (spectral analysis) for the specified axis
    Args:
        1) df_arg - a dataframe that contains the required data
        2) signal_column - a column that contains data for the specified axis
        3) sampling_rate - actual sampling rate of your dataset (samples per second)
        4) is_divided_by_g - the content of the column was divided by the free fall acceleration g (g=9.81)
    """
    signal = df_arg[signal_column]
    signal = np.array(signal)
    
    if is_divided_by_g:
        signal = signal * 9.81
        
    fft_result = fft(signal)

    # Calculate the corresponding frequencies for the FFT result
    freqs = fftfreq(len(signal), 1 / sampling_rate)

    # Plot the magnitude of the frequency spectrum
    plt.figure(figsize=(10, 6))
    plt.plot(freqs, np.abs(fft_result))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.title(f'Frequency Spectrum Analysis {signal_column}')
    plt.grid()

    plt.show()

### Frequency analysis of the signal spectrum for each activity

In [ ]:
def display_activity_freq_spectrum_one_axes(df, activity_name, axes_name, sampling_rate, is_divided_by_g=False, x_lim=10, color='blue'):
    """ Function to display the frequency spectrum of the specified type of activity on the OX, OY and OZ axes for the specified device 
    (accelerometer or gyroscope)
    Args:
        1) df - a dataframe that contains the results of accelerometer and gyroscope measurements
        2) activity_name - selected activity type (among the values of the 'activity' column of df)
        3) axes_name - the column name of the dataframe df that contain the measurement results for the OX, OY, or OZ axes
        4) sampling_rate - actual sampling rate of your dataset (samples per second)
        5) is_divided_by_g - True if the content of the column was divided by the free fall acceleration g (g=9.81 m/s^2)
        6) x_lim - the limits of the graph along the OX axis are [-x_lim; x_lim]
        7) color - the color of the graph
    """
    activity_data = df[df['activity'] == activity_name]
    
    signal = np.array(activity_data[axes_name])
    
    if is_divided_by_g:
        signal *= 9.81
        
    fft_result = fft(signal)

    fft_freqs = fftfreq(len(activity_data), 1/sampling_rate)

    fig = plt.figure(figsize=(12, 5))
    axes = fig.add_axes([0.1, 0.1, 1, 1])
    axes.plot(fft_freqs, np.abs(fft_result), color=color)
    axes.set_title(f'{activity_name} {axes_name}')

    axes.set_xlabel('Frequency (Hz)')
    axes.set_xlim(-x_lim, x_lim)
    axes.set_ylabel('Amplitude')
    axes.grid(alpha=0.5)

    plt.show()

    
def display_activity_freq_spectrum(df, activity_name, axes_names, sampling_rate, is_divided_by_g=False, x_lim=10):
    """ Function to display the frequency spectrum of the specified type of activity on the OX, OY and OZ axes for the specified device 
    (accelerometer or gyroscope)
    Args:
        1) df - a dataframe that contains the results of accelerometer and gyroscope measurements
        2) activity_name - selected activity type (among the values of the 'activity' column of df)
        3) axes_names - an array containing the column names of the dataframe df that contain the measurement results for the OX, OY, and OZ axes, 
        respectively
        4) sampling_rate - actual sampling rate of your dataset (samples per second)
        5) is_divided_by_g - True if the content of the column was divided by the free fall acceleration g (g=9.81 m/s^2)
        6) x_lim - the limits of the graph along the OX axis are [-x_lim; x_lim]
    """
    activity_data = df[df['activity'] == activity_name]
    
    signal_x = np.array(activity_data[axes_names[0]])
    signal_y = np.array(activity_data[axes_names[1]])
    signal_z = np.array(activity_data[axes_names[2]])
    
    if is_divided_by_g:
        signal_x *= 9.81
        signal_y *= 9.81
        signal_z *= 9.81
        
    fft_x = fft(signal_x)
    fft_y = fft(signal_y)
    fft_z = fft(signal_z)

    fft_freqs = fftfreq(len(activity_data), 1/sampling_rate)

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    axes[0].plot(fft_freqs, np.abs(fft_x), label='X', color='blue')
    axes[0].set_title(f'{activity_name} {axes_names[0]}')

    axes[1].plot(fft_freqs, np.abs(fft_y), label='Y', color='yellow')
    axes[1].set_title(f'{activity_name} {axes_names[1]}')

    axes[2].plot(fft_freqs, np.abs(fft_z), label='Z', color='green')
    axes[2].set_title(f'{activity_name} {axes_names[2]}')

    for ax in axes:
        ax.set_xlabel('Frequency (Hz)')
        ax.set_xlim(-x_lim, x_lim)
        ax.set_xticks([i for i in range(-x_lim, x_lim+1, 2)])
        ax.set_ylabel('Amplitude')
        ax.grid(alpha=0.5)
        ax.legend()

    plt.tight_layout()
    plt.show()

# Reading and Preparing data

In [ ]:
df = read_clear_df(filepath='original_data/2023-08-01.21_21_11_Test_1.csv', skiprows=1, delimiter=';', columns_num=7)
df.head()

In [ ]:
df.info()

In [ ]:
print(f"Measurement start time = {df.iloc[0]['time']}")
print(f"Measurement end time = {df.iloc[-1]['time']}")

In [ ]:
time_diffs = df['time'].diff()
freq = 1.0 / time_diffs.mean()
print(f"Measurement time = {df.iloc[-1]['time'] - df.iloc[0]['time']} s")
print(f"Number of measurements (number of rows in the data set) = {len(df)}")
print(f"Frequency of measurement = {freq} Hz")

# Visualise accelerometer and gyroscope behaviour (Raw data)

### Visualise accelerometer results

In [ ]:
display_three_axes(df=df, x='time', y=['gFx', 'gFy', 'gFz'], title='Time Dependence of Linear Acceleration (Raw Data)', 
                x_label='Time, s', y_label='Linear acceleration / g')

In [ ]:
display_three_axes_sep(df=df, x='time', y=['gFx', 'gFy', 'gFz'], title='Accelerometer operation (Raw Data)', x_label='time', 
                       y_labels=['AccX, m/s^2', 'AccY, m/s^2', 'AccZ, m/s^2'])

### Visualise gyroscope results

In [ ]:
display_three_axes(df=df, x='time', y=['wx', 'wy', 'wz'], title='Time Dependence of Angular Velocity (Raw Data)', 
                x_label='Time, s', y_label='Angular velocity, rad/s')

In [ ]:
display_three_axes_sep(df=df, x='time', y=['wx', 'wy', 'wz'], title='Gyroscope operation (Raw Data)', x_label='time', 
                       y_labels=['GyrX, rad/s^2', 'GyrY, rad/s^2', 'GyrZ, rad/s^2'])

# Check the stability of the data sampling period and frequency

In [ ]:
time_measurement_df = get_measurement_time_df(df)
time_measurement_df.head()

In [ ]:
df.head(6)

### Check the last 5 items

In [ ]:
time_measurement_df[-5:]

In [ ]:
df[-5:]

## Check the measurement time distribution

In [ ]:
display_time_distribution(df=time_measurement_df, x='measurement_time')

In [ ]:
time_measurement_df[time_measurement_df['measurement_time'] > 0.015]

In [ ]:
time_measurement_df[time_measurement_df['measurement_time'] > 0.025]

## Investigate the measurement_time distribution relative to the measurement time and number of measurement

In [ ]:
freq = 1.0 / time_measurement_df['measurement_time'].mean()
print(f"freq = {freq} Hz")

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(time_measurement_df['start_time'], time_measurement_df['measurement_time'])
plt.title(f'Stability of Data Collection (Raw)\nAverage frequency: {freq:.3f} Hz')
plt.xlabel('time of measurement, s')
plt.ylabel('time to measure, s')

In [ ]:
plt.figure(figsize=(12, 3))
plt.title(f'Stability of Data Collection (Raw)\nAverage frequency: {freq:.3f} Hz')
plt.ylabel('measurement time, s')
sns.lineplot(x=range(len(time_measurement_df['measurement_time'])), y=time_measurement_df['measurement_time'])
plt.show()

## Trying to discard too long time intervals between measurements (large values of 'measurement_time')

In [ ]:
time_measurement_df['lost_data'] = time_measurement_df['measurement_time'].apply(lambda time: 1 if time > 0.012 else 0)
print(f"len(received_data) = {len(time_measurement_df[time_measurement_df['lost_data'] == 0])}")
print(f"len(lost_data) = {len(time_measurement_df[time_measurement_df['lost_data'] == 1])}")
print(f"Percentage of data lost to data received: " 
    f"{np.round((len(time_measurement_df[time_measurement_df['lost_data'] == 1]) / len(time_measurement_df[time_measurement_df['lost_data'] == 0]) * 100), 2)}%")
print(f"Percentage of data lost to all data: " 
      f"{np.round((len(time_measurement_df[time_measurement_df['lost_data'] == 1]) / len(time_measurement_df['lost_data']) * 100), 2)}%")

## Display the result of filtering measurement time

In [ ]:
display_data_collection_losses_measurement_number(df=time_measurement_df)

In [ ]:
display_data_collection_losses_measurement_time(df=time_measurement_df)

In [ ]:
display_data_collection_stability_measurement_number(df=time_measurement_df)

In [ ]:
display_data_collection_stability_measurement_time(df=time_measurement_df)

In [ ]:
display_data_collection_stability(df=time_measurement_df)

# Frequency analysis of the signal spectrum

In [ ]:
# Add 'activity' column 
add_activity_column(path='labeled_data/Test_1_labeled.csv', df=df, df_merge_col='time')
df.head()

## Analyse data in general

### Analyse data for accelerometer

In [ ]:
signal_x = df['gFx'] * 9.81
signal_x = np.array(signal_x)
print(type(signal_x))
print(signal_x[30:35])

fft_result = fft(signal_x)

# Calculate the corresponding frequencies for the FFT result
sampling_rate = 806.325
freqs = fftfreq(len(signal_x), 1 / sampling_rate)

# Plot the magnitude of the frequency spectrum
plt.figure(figsize=(10, 6))
plt.plot(freqs, np.abs(fft_result))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('Frequency Spectrum Analysis')
plt.grid()

plt.show()

In [ ]:
show_freq_distribution(df_arg=df, signal_column='gFx', sampling_rate=806.325, is_divided_by_g=True)

##### Analysing the code above step by step

In [ ]:
signal_x = df['gFx'] * 9.81
signal_x = np.array(signal_x)

fft_result = fft(signal_x)

print(f'signal_x[30:35] = {signal_x[30:35]}')
print(f'fft_result[30:35] = {fft_result[30:35]}')

In [ ]:
# Calculate the corresponding frequencies for the FFT result
sampling_rate = 806.325
freqs = fftfreq(len(signal_x), 1 / sampling_rate)
print(f'freqs[30:35] = {freqs[30:35]}')

In [ ]:
print(f'np.abs(fft_result)[30:35] = {np.abs(fft_result)[30:35]}')

In [ ]:
len(freqs)

In [ ]:
len(fft_result)

In [ ]:
counter = 0
for index, freq in enumerate(freqs):
    if freq > 300:
        if counter < 10:
            print(f"time = {df.iloc[index]['time']}; {df.iloc[index]['gFx']} : {freq}")
        counter += 1
        
print(f"len(freq > 300) = {counter}")

In [ ]:
# Plot the magnitude of the frequency spectrum
plt.figure(figsize=(10, 6))
plt.plot(df['time'], freqs)
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('Frequency Spectrum Analysis')
plt.xticks([ i for i in range(0, 120, 10)])
plt.grid()

plt.show()

In [ ]:
# Plot the magnitude of the frequency spectrum
plt.figure(figsize=(10, 6))
plt.plot(df['time'], df['gFx'])
plt.xlabel('Time (s)')
plt.ylabel('AccX')
plt.title('Accelerometer axis=OX')
plt.xticks([ i for i in range(0, 120, 10)])
plt.grid()

plt.show()

#### Analyse data for y-axis

In [ ]:
signal_y = df['gFy'] * 9.81
signal_y = np.array(signal_y)
print(type(signal_y))
print(signal_y[:5])

fft_result = fft(signal_y)

# Calculate the corresponding frequencies for the FFT result
sampling_rate = 806.325
freqs = fftfreq(len(signal_y), 1 / sampling_rate)

# Plot the magnitude of the frequency spectrum
plt.figure(figsize=(10, 6))
plt.plot(freqs, np.abs(fft_result))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('Frequency Spectrum Analysis')
plt.grid()

plt.show()

In [ ]:
show_freq_distribution(df_arg=df, signal_column='gFy', sampling_rate=806.325, is_divided_by_g=True)

#### Analyse data for z-axis

In [ ]:
signal_z = df['gFz'] * 9.81
signal_z = np.array(signal_z)
print(type(signal_z))
print(signal_z[:5])

fft_result = fft(signal_z)

# Calculate the corresponding frequencies for the FFT result
sampling_rate = 806.325
freqs = fftfreq(len(signal_z), 1 / sampling_rate)

# Plot the magnitude of the frequency spectrum
plt.figure(figsize=(10, 6))
plt.plot(freqs, np.abs(fft_result))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('Frequency Spectrum Analysis')
plt.grid()

plt.show()

In [ ]:
show_freq_distribution(df_arg=df, signal_column='gFz', sampling_rate=806.325, is_divided_by_g=True)

### Analyse data for gyroscope

#### Analyse data for x-axis

In [ ]:
show_freq_distribution(df_arg=df, signal_column='wx', sampling_rate=806.325, is_divided_by_g=False)

#### Analyse data for y-axis

In [ ]:
show_freq_distribution(df_arg=df, signal_column='wy', sampling_rate=806.325, is_divided_by_g=False)

#### Analyse data for z-axis

In [ ]:
show_freq_distribution(df_arg=df, signal_column='wz', sampling_rate=806.325, is_divided_by_g=False)

## Perform a frequency spectrum analysis for each type of activity

### Visualize frequency spectrum for each type of activity

In [ ]:
display_activity_freq_spectrum(df=df, activity_name='Squat', axes_names=['gFx', 'gFy', 'gFz'], sampling_rate=806.3249042011819, is_divided_by_g=True)

#### Frequency analysis of the signal spectrum of various types of physical activities for the accelerometer

In [ ]:
activities = df['activity'].unique()
for activity in activities:
    display_activity_freq_spectrum(df=df, activity_name=activity, axes_names=['gFx', 'gFy', 'gFz'], 
                                   sampling_rate=1.0 / df['time'].diff().mean(), 
                                   is_divided_by_g=True)

#### Frequency analysis of the signal spectrum of various types of physical activities for the gyroscope

In [ ]:
activities = df['activity'].unique()
for activity in activities:
    display_activity_freq_spectrum(df=df, activity_name=activity, axes_names=['wx', 'wy', 'wz'], 
                                   sampling_rate=1.0 / df['time'].diff().mean())

# Save dataframe in normal .csv file (with ',' as delimiter)

In [ ]:
df.to_csv('correct_csv_data/Test_1.csv', sep=',', index=False)

In [ ]:
## Read data to check if it was saved correctly
df_test = pd.read_csv('correct_csv_data/Test_1.csv')
df_test.head()

In [ ]:
df_test.info()

## Create smaller dataset to check in Label Studio

In [ ]:
small_df = df[0:-1:100]
len(small_df)

In [ ]:
display_three_axes(df=small_df, x='time', y=['gFx', 'gFy', 'gFz'], title='Time Dependence of Linear Acceleration (Raw Data)', 
                x_label='Time, sec', y_label='Linear acceleration / g, m/s^2')

In [ ]:
small_df.to_csv('correct_csv_data/Test_1_small.csv', sep=',', index=False)

### Check the content of the 'Test_1_small.csv' file

In [ ]:
## Read data to check if it was saved correctly
small_df = pd.read_csv('correct_csv_data/Test_1_small.csv')
small_df.head()

In [ ]:
len(small_df)

In [ ]:
display_three_axes(df=small_df, x='time', y=['gFx', 'gFy', 'gFz'], title='Time Dependence of Linear Acceleration (Raw Data)', 
                x_label='Time, sec', y_label='Linear acceleration / g, m/s^2')

# Demonstration

In [ ]:
display_three_axes(df=df, x='time', y=['gFx', 'gFy', 'gFz'], title='Time Dependence of Linear Acceleration (Raw Data)', 
                x_label='Time, s', y_label='Linear acceleration / g')
display_three_axes(df=df, x='time', y=['wx', 'wy', 'wz'], title='Time Dependence of Angular Velocity (Raw Data)', 
                x_label='Time, s', y_label='Angular velocity, rad/s')